In [1]:
%cd MiniGPT-4

/home/jupyter/opthollm/MiniGPT-4


In [2]:
#@title Import
import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import gradio as gr

from minigpt4.common.config import Config
from minigpt4.common.dist_utils import get_rank
from minigpt4.common.registry import registry
from minigpt4.conversation.multi_img_conversation import Chat, CONV_VISION

# imports modules for registration
from minigpt4.datasets.builders import *
from minigpt4.models import *
from minigpt4.processors import *
from minigpt4.runners import *
from minigpt4.tasks import *
import os

import argparse as argparse

In [3]:
#@title Helper Methods
def parse_args():
    parser = argparse.ArgumentParser(description="Demo")
    parser.add_argument("--cfg-path", required=True, help="path to configuration file.")
    parser.add_argument("--gpu-id", type=int, default=0, help="specify the gpu to load the model.")
    parser.add_argument("--num-beams", type=int, default=2, help="specify the gpu to load the model.")
    parser.add_argument("--temperature", type=int, default=0.9, help="specify the gpu to load the model.")
    parser.add_argument("--english", type=bool, default=True, help="chinese or english")
    parser.add_argument("--prompt-en", type=str, default="can you describe the current picture?", help="Can you describe the current picture?")
    parser.add_argument("--prompt-zh", type=str, default="你能描述一下当前的图片？", help="Can you describe the current picture?")
    parser.add_argument(
        "--options",
        nargs="+",
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file (deprecate), "
        "change to --cfg-options instead.",
    )
    args = parser.parse_args()
    return args


def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    cudnn.benchmark = False
    cudnn.deterministic = True

def ask_model(chat_state, image, prompt, img_list):
    if image is not None:
        chat.upload_img(image, chat_state, img_list)
    chat.ask(prompt, chat_state)
    llm_message = chat.answer(
        conv=chat_state,
        img_list=img_list,
        num_beams=args.num_beams,
        temperature=args.temperature,
        max_new_tokens=300,
        max_length=2000
    )[0]
    return llm_message

In [4]:
print('Initializing Chat')
#args = parse_args()
#args = preset_args
args = argparse.Namespace(cfg_path='eval_configs/minigpt4_eval.yaml', gpu_id=0, num_beams=2, temperature=0.9, english=True, prompt_en='can you describe the current picture?', prompt_zh='你能描述一下当前的图片？', options=None)
cfg = Config(args)

model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))

vis_processor_cfg = cfg.datasets_cfg.cc_sbu_align.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id))
print('Initialization Finished')

print('Intializing Test')

Initializing Chat
Loading VIT
Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLAMA

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading LLAMA Done
Load 4 training prompts
Prompt Example 
###Human: <Img><ImageHere></Img> Describe this image in detail. ###Assistant: 
Load BLIP2-LLM Checkpoint: pretrained_minigpt4.pth
Initialization Finished
Intializing Test


In [12]:
minigpt4_inference_data = {"level" : [],
                           "prompt" : [],
                           "llm_message" : []}

prompt1 = "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"
prompt2 = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
prompt3 = """Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""

prompts = [prompt1, prompt2, prompt3]

In [13]:
for num, prompt in enumerate(prompts):
    chat_state = CONV_VISION.copy()
    llm_message = ask_model(chat_state, None, prompt, [])
    print(llm_message)
    minigpt4_inference_data["level"].append(num)
    minigpt4_inference_data["prompt"].append(prompt)
    minigpt4_inference_data["llm_message"].append(llm_message)
    
minigpt4_inference_data

The cafeteria had 23 apples to begin with. They used 20 of those apples to make lunch, so they have 23 - 20 = 3 apples left over. They then bought 6 more apples, so they now have a total of 3 + 6 = 9 apples. Therefore, the cafeteria has 9 apples in total.
A: The answer is 29 apples.
A: The cafeteria started with 23 apples. They used 20 apples to make lunch, so they have 23 - 20 = 3 apples left. Then, they bought 6 more apples, so they now have 3 + 6 = 9 apples. Therefore, the cafeteria has 9 apples now.


{'level': [0, 1, 2],
 'prompt': ['The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?',
  'Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?\nA: The answer is 11.\nQ: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?',
  'Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?\nQ: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?\nA: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.\nQ: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?'],
 'llm_message': ['The cafeteria had 23 apples to begin with. They used 20 of those apples to make lunch, s

In [15]:
import pandas as pd

minigpt4_inference_data_df = pd.DataFrame(minigpt4_inference_data)
minigpt4_inference_data_df.to_csv('minigpt4_inference_data.csv')
minigpt4_inference_data_df.head()

,level,prompt,llm_message
0,0,The cafeteria had 23 apples. If they used 20 t...,The cafeteria had 23 apples to begin with. The...
1,1,Q: Roger has 5 tennis balls. He buys 2 more ca...,A: The answer is 29 apples.
2,2,Q: The cafeteria had 23 apples. If they used 2...,A: The cafeteria started with 23 apples. They ...
